In [97]:
from os import listdir, path, getcwd, mkdir, environ
import shutil
import cv2
import glob
from PIL import Image
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [71]:
train_images_path = "./images/train"
test_images_path = "./images/test"
emotions = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']
emotions_map = {'Anger': 0, 'Disgust': 1, 'Fear': 2, 'Happiness':3, 'Sadness':4, 'Surprise':5}

In [44]:
filelist = []
for emotion in sorted(listdir(train_images_path)):
    if not path.isdir(path.join(train_images_path, emotion)):
        continue
    print("===================")
    print("Processing emotion: " + emotion)
    filelist = filelist + glob.glob(path.join(train_images_path, emotion, '*.jpg'))

Processing emotion: Anger
Processing emotion: Disgust
Processing emotion: Fear
Processing emotion: Happiness
Processing emotion: Sadness
Processing emotion: Surprise


In [74]:
target = []
for filename in filelist:
    target.append(emotions_map[filename.split('/')[-1].split('train')[1].split('0')[0]])

y = np.array(target)
y = to_categorical(y, num_classes=6 )

In [96]:
data = np.array([np.array(Image.open(img)) for img in filelist])
data = data.astype(np.float)
data /= 255
data.shape

(11965, 96, 96)

In [101]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=12)